![banner](./images/banner.png "banner")

# Modelo de aprendizaje automático para la predicción del Ratio Internacional Normalizado (INR) en pacientes bajo terapia con Antagonistas de la Vitamina K

## Tabla de contenidos

- [1. Descripción del proyecto](#1-descripción-del-proyecto)
  - [1.1 Objetivos](#11-objetivos)
  - [1.2 Datos](#12-datos)
  - [1.3 Software](#13-software)

- [2. Exploración y visualización de datos](#2-exploracion-y-visualizacion-de-datos)
  - [2.1 Carga de datos](#21-carga-de-datos)
  - [2.2 Generación del csv](#22-generacion-del-csv)
    - [2.2.1 Simplificación de las variables y pivotación](#221-simplificacion-de-las-variables-y-pivotacion)
  - [2.3 Limpieza de datos](#23-limpieza-de-datos)
      - [2.3.1 Valores faltantes y types](#221-simplificacion-de-las-variables-y-pivotacion)
      - [2.3.2 Duplicados](#221-simplificacion-de-las-variables-y-pivotacion)
      - [2.3.3 Eliminación de variables redundantes o irrelevantes](#221-simplificacion-de-las-variables-y-pivotacion)
      - [2.3.4 Rangos y estadistica basica](#221-simplificacion-de-las-variables-y-pivotacion)
      - [2.3.4 Valores únicos](#221-simplificacion-de-las-variables-y-pivotacion)
  - [2.4 Feature Importance y análisis de la variable target](#24-feature-importance-y-analisis-de-la-variable-target)
    - [2.4.1 Oversampling data](#241-oversampling-data)


## 1. Descripción del proyecto


Este proyecto aborda un problema de **predicción de un valor continuo**. Se trata de una tarea supervisada: el modelo aprende a partir de un conjunto amplio de ejemplos donde todas las columnas, incluida la variable target, están completas. Una vez entrenado, aplicamos el modelo a nuevas filas en las que falta precisamente ese valor que queremos estimar. Las columnas con información siempre disponible se denominan características o features.

La variable target puede tomar distintos valores, incluyendo valores inferiores a 1.0 o superiores a 3.0 correspondientes a las mediciones del INR (Ratio Internacional Normalizado). Al ser un valor continuo, la **metodología adecuada es la regresión**.

Si los datos no vinieran etiquetados, si no conociéramos el target desde el inicio, sería necesario recurrir a técnicas de aprendizaje no supervisado, donde el modelo identifica patrones o grupos sin una referencia previa.

En nuestro caso, ajustaremos diferentes modelos de regresión sobre datos de pacientes en tratamiento anticoagulante y elegiremos el que ofrezca el mejor rendimiento. Para evaluar de forma fiable el comportamiento del modelo, dividimos el conjunto de datos en tres partes:

- Conjunto de entrenamiento: contiene los datos completos, incluidos los valores reales del target. Sirven para que el modelo aprenda las relaciones entre las características y la variable objetivo.

- Conjunto de validación: simulamos datos “nuevos” separando previamente el target y ocultándolo al modelo. Tras el entrenamiento, comparamos sus predicciones con los valores reales reservados. Como este conjunto se usa repetidamente para ajustar parámetros, existe el riesgo de que el modelo se adapte demasiado a él y pierda capacidad de generalización (sobreajuste o overfitting).

- Conjunto de prueba: incluye ejemplos completamente nuevos para el modelo entrenado y validado. Es la comprobación final del rendimiento y nos permite estimar cómo respondería ante datos reales que aún no hemos observado.


#### Este proyecto contendrá:

- La preparación y limpieza de los datos para que puedan ser utilizados por los modelos de predicción. `Exploration_and_Classification.ipynb`

- La selección de la métrica de rendimiento más adecuada para evaluar la calidad de las predicciones del modelo. `Exploration_and_Classification.ipynb`

- El entrenamiento de los modelos de regresión, ajustando sus parámetros para optimizar la métrica seleccionada. `ModelTraining_and_Conclusions.ipynb`

- La aplicación de los modelos a nuevos datos para generar predicciones de la variable objetivo. `ModelTraining_and_Conclusions.ipynb`

- La evaluación del rendimiento final del modelo comparando sus predicciones con los valores reales de la variable objetivo. `ModelTraining_and_Conclusions.ipynb`

- Un análisis de explicabilidad para comprender cómo el modelo toma sus decisiones y cuáles características influyen más en las predicciones. `XAI.ipynb`

- La cuantificación de la incertidumbre asociada a las predicciones, para entender la confianza que podemos tener en los resultados del modelo. `UQ.ipynb`

### 1.1 Objetivos

El objetivo de este proyecto es **predecir el valor del INR de pacientes bajo tratamiento con anticoagulantes** utilizando variables analíticas, hábitos de vida y otros factores de riesgo relevantes. Este análisis permite identificar patrones que influyen en el INR, facilitando la detección temprana de posibles complicaciones y apoyando la prevención de diagnósticos complejos.

### 1.2 Datos

El conjunto de datos utilizado para entrenar y evaluar nuestro modelo proviene de **Synthea** (https://synthetichealth.github.io/synthea/), un software de código abierto desarrollado en Estados Unidos por The MITRE Corporation. 

Este programa genera de manera sintética historias clínicas completas de pacientes, incluyendo información demográfica, resultados de pruebas de laboratorio, medicación, hábitos de vida y otros factores clínicos relevantes. Gracias a estos datos sintéticos, podemos trabajar con escenarios clínicos muy realistas sin enfrentarnos a las complejidades legales y burocráticas relacionadas con la protección de datos de pacientes reales. Además, nos permite evaluar la calidad y utilidad de los datos sintéticos, que cada vez están cobrando mayor relevancia en la investigación médica y el desarrollo de modelos predictivos.

Para ejecutar el programa ejecutremos `java -jar synthea.jar` este procesará los valores que se indiquen en el archivo de configuración **synthea.properties** indicaremos lo siguiente:

- default_population = 5.000 -> cantida de pacientes
- exporter.csv.export = true  -> se exportaran los datos a ficheros csv
- exporter.csv.append_mode = true   -> si se ejecuta otra vez el archivo se actualizaran los datos del csv
- generate.modules = generate.modules = cardiovascular_disease, atrial_fibrillation, venous_thromboembolism, cerebrovascular_accident, heart_failure, medications_anticoagulants, inr_monitoring -> permite priorizar la generación de pacientes con estas patologias.


Añadiremos los módulos `inr_monitoring` y `medications_anticoagulants` en generate.modules, lo que nos permitirá generar un conjunto de pacientes más amplio y específico, centrado en aquellos que están bajo tratamiento con anticoagulantes. El módulo medications_anticoagulants asegura que los pacientes reciban distintos tipos de medicación anticoagulante, mientras que inr_monitoring simula de manera realista el seguimiento de sus niveles de INR a lo largo del tiempo.

Al ejecutar Synthea, los datos generados se exportan automáticamente a la carpeta output. Las historias de los pacientes se organizan en diferentes archivos CSV, cada uno correspondiente a un módulo específico del historial clínico, que puede incluir desde alergias hasta medicaciones y cobertura de seguro médico. Para el análisis de este proyecto, nos centraremos únicamente en los datos contenidos en los archivos `observations.csv`, `patients.csv` y `medications.csv`, ya que estos proporcionan la información necesaria sobre resultados de laboratorio, características demográficas y tratamientos farmacológicos de los pacientes.



#### Observations
| | Column Name | Data Type | Required? | Description |
|-|-------------|-----------|-----------|-------------|
| | Date | iso8601 UTC Date (`yyyy-MM-dd'T'HH:mm'Z'`) | `true` | The date and time the observation was performed. |
| | Patient | UUID | `true` | Foreign key to the Patient. |
| | Encounter | UUID | `true` | Foreign key to the Encounter where the observation was performed. |
| | Category | String | `false` | Observation category. |
| | Code | String | `true` | Observation or Lab code from LOINC |
| | Description | String | `true` | Description of the observation or lab. |
| | Value | String | `true` | The recorded value of the observation. Often numeric, but some values can be verbose, for example, multiple-choice questionnaire responses. |
| | Units | String | `false` | The units of measure for the value. |
| | Type | String | `true` | The datatype of `Value`: `text` or `numeric` |


#### Patients
| | Column Name | Data Type | Required? | Description |
|-|-------------|-----------|-----------|-------------|
| | Id | UUID | `true` | Primary Key. Unique Identifier of the patient. |
| | BirthDate | Date (`YYYY-MM-DD`) | `true` | The date the patient was born. |
| | DeathDate | Date (`YYYY-MM-DD`) | `false` | The date the patient died. |
| | SSN | String | `true` | Patient Social Security identifier. |
| | Drivers | String | `false` | Patient Drivers License identifier. |
| | Passport | String | `false` | Patient Passport identifier. |
| | Prefix | String | `false` | Name prefix, such as `Mr.`, `Mrs.`, `Dr.`, etc. |
| | First | String | `true` | First name of the patient. |
| | Middle | String | `false` | Middle name of the patient. |
| | Last | String | `true` | Last or surname of the patient. |
| | Suffix | String | `false` | Name suffix, such as `PhD`, `MD`, `JD`, etc. |
| | Maiden | String | `false` | Maiden name of the patient. |
| | Marital | String | `false` | Marital Status. `M` is married, `S` is single. Currently no support for divorce (`D`) or widowing (`W`) |
| | Race | String | `true` | Description of the patient's primary race. |
| | Ethnicity | String | `true` | Description of the patient's primary ethnicity. |
| | Gender | String | `true` | Gender. `M` is male, `F` is female. |
| | BirthPlace | String | `true` | Name of the town where the patient was born. |
| | Address | String | `true` | Patient's street address without commas or newlines. |
| | City | String | `true` | Patient's address city. |
| | State | String | `true` | Patient's address state. |
| | County | String | `false` | Patient's address county. |
| | FIPS County Code | String | `false` | Patient's FIPS county code. |
| | Zip | String | `false` | Patient's zip code. |
| | Lat | Numeric | `false` | Latitude of Patient's address. |
| | Lon | Numeric | `false` | Longitude of Patient's address. |
| | Healthcare_Expenses | Numeric | `true` | The total lifetime cost of healthcare to the patient (i.e. what the patient paid). |
| | Healthcare_Coverage | Numeric | `true` | The total lifetime cost of healthcare services that were covered by Payers (i.e. what the insurance company paid). |
| | Income | Numeric | `true` | Annual income for the Patient |


#### Medications
| | Column Name | Data Type | Required? | Description |
|-|-------------|-----------|-----------|-------------|
| | Start | iso8601 UTC Date (`yyyy-MM-dd'T'HH:mm'Z'`) | `true` | The date and time the medication was prescribed. |
| | Stop | iso8601 UTC Date (`yyyy-MM-dd'T'HH:mm'Z'`) | `false` | The date and time the prescription ended, if applicable. |
| :old_key: | Patient | UUID | `true` | Foreign key to the Patient. |
| :old_key: | Payer | UUID | `true` | Foreign key to the Payer. |
| :old_key: | Encounter | UUID | `true` | Foreign key to the Encounter where the medication was prescribed. 
| | Code | String | `true` | Medication code from RxNorm. |
| | Description | String | `true` | Description of the medication. |
| | Base_Cost | Numeric | `true` | The line item cost of the medication. |
| | Payer_Coverage | Numeric | `true` | The amount covered or reimbursed by the Payer. |
| | Dispenses | Numeric | `true` | The number of times the prescription was filled. |
| | TotalCost | Numeric | `true` | The total cost of the prescription, including all dispenses. |
| | ReasonCode | String | `false` | Diagnosis code from SNOMED-CT specifying why this medication was prescribed. |
| | ReasonDescription | String | `false` | Description of the reason code. |


La variable **target** que utilizaremos en este análisis se encuentra dentro del archivo observations.csv y corresponde al valor INR de los pacientes bajo tratamiento con anticoagulantes. Esta variable es continua y representa el riesgo de que un paciente desarrolle complicaciones cardíacas, interpretándose de la siguiente manera:

- **INR < 2.0:** el paciente presenta riesgo de padecer una cardiopatía.

- **INR entre 2.0 y 3.0:** el paciente se encuentra dentro del rango terapéutico, por lo que no presenta riesgo significativo de cardiopatía.

- **INR > 3.0:** aunque el INR es elevado, el paciente no se considera en riesgo de cardiopatía para los efectos de este análisis, aunque sí puede indicar riesgo de hemorragias u otras complicaciones.

Esta definición nos permite utilizar el INR como variable objetivo para entrenar modelos de predicción que puedan estimar el riesgo clínico basado en los datos del paciente.

## 1.3 Software

Importamos las siguientes librerías:

In [1]:
import pandas as pd
import numpy as np

#utils files
from utils.cleaner_category_match import *
from utils.cleaner_shorter_categories import *

# data exploration and preparation  
from sklearn.feature_selection import mutual_info_classif

# plotting and displaying in the notebook
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import display
from sklearn import tree

#SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

#model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression


# 2. Exploración y visualización de datos

## 2.1 Carga de datos

Vamos a cargar los datos del fichero `patients.csv`:

In [2]:
df_patients = pd.read_csv("data_synthea/patients.csv")
df_patients.head()
#df_patients.info()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,MIDDLE,LAST,...,CITY,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME
0,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,2016-03-11,NaN,999-57-5423,NaN,NaN,NaN,Norris589,Bill567,Hermiston71,...,Wareham,Massachusetts,Plymouth County,NaN,0,41.766794,-70.652862,1543.07,17566.83,10551
1,a48ad472-6249-d9df-9352-6d04658dc834,1995-01-05,NaN,999-62-7499,S99923922,X56579640X,Mr.,Daron260,Denver542,Mohr916,...,Dartmouth,Massachusetts,Bristol County,NaN,0,41.521893,-70.941710,12243.27,725326.99,14047
2,6875adfb-1f3e-1e01-2c4d-1dc19a5c04bc,2018-07-02,NaN,999-27-9058,NaN,NaN,NaN,Anja508,NaN,Bergstrom287,...,Gloucester,Massachusetts,Essex County,25009.0,1930,42.630311,-70.673016,1664.88,14130.35,5879
3,56689f99-ca4d-67eb-12d6-5b0f0dc45b59,2014-11-04,NaN,999-29-5447,NaN,NaN,NaN,Jeneva675,NaN,Gerhold939,...,Quincy,Massachusetts,Norfolk County,25021.0,2186,42.233034,-71.052074,40323.31,4448.38,905888
4,9169a441-7be6-032e-690b-0c09c868c9e7,2002-08-22,NaN,999-67-4934,S99982373,X5968327X,Ms.,Sonia106,Carlota980,Laboy63,...,Malden,Massachusetts,Middlesex County,25017.0,2155,42.449900,-71.059329,72881.03,3817.85,130763


El conjunto de datos contiene información de **5.693 pacientes y 28 variables que describen sus características básicas y demográficas**, como edad, género, etnia, estado civil, lugar de nacimiento e ingresos. Incluye además datos administrativos (por ejemplo, número de seguro social o pasaporte) y médicos (como cobertura de salud). En general, ofrece una visión completa del perfil socio-demográfico de cada paciente.

Vamos a cargar los datos del fitchero `medications.csv`

In [3]:
df_medications = pd.read_csv("data_synthea/medications.csv")
df_medications.head()
#df_medications.info()

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2016-05-15T04:16:23Z,2016-05-26T21:16:23Z,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,df166300-5a78-3502-a46a-832842197811,ae1acd06-3eb7-c5c5-518b-0d684699d637,834061,Penicillin V Potassium 250 MG Oral Tablet,71.14,21.14,1,71.14,43878008.0,Streptococcal sore throat (disorder)
1,2018-05-30T11:16:23Z,2018-06-13T11:16:23Z,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,df166300-5a78-3502-a46a-832842197811,ae1acd06-3eb7-c5c5-f865-967d31a68c55,313820,Acetaminophen 160 MG Chewable Tablet,47.21,0.00,1,47.21,NaN,NaN
2,2019-11-20T11:57:35Z,2020-02-29T11:57:35Z,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,df166300-5a78-3502-a46a-832842197811,ae1acd06-3eb7-c5c5-33e6-7ab6ee0e09a1,313820,Acetaminophen 160 MG Chewable Tablet,45.86,0.00,3,137.58,NaN,NaN
3,2023-03-30T12:29:17Z,2023-05-20T12:29:17Z,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,df166300-5a78-3502-a46a-832842197811,ae1acd06-3eb7-c5c5-0024-f3f732c42c2b,198405,Ibuprofen 100 MG Oral Tablet,141.13,0.00,1,141.13,NaN,NaN
4,1996-07-02T16:13:27Z,NaN,a48ad472-6249-d9df-9352-6d04658dc834,df166300-5a78-3502-a46a-832842197811,a48ad472-6249-d9df-312f-fd3804195749,665078,Loratadine 5 MG Chewable Tablet,76.44,26.44,357,27289.08,NaN,NaN


El conjunto de datos contiene información de **265.954 medicaciones asociadas a pacientes y 13 variables**, que incluyen detalles sobre fechas de atención, identificación del paciente, tipo de cobertura, códigos y descripciones de las medicaciones, costos asociados y dispensas.

Vamos a cargar los datos del fichero `observations.csv`:

In [4]:
df_observations = pd.read_csv("data_synthea/observations.csv")
#df_observations.head()
df_observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4144942 entries, 0 to 4144941
Data columns (total 9 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   DATE         object
 1   PATIENT      object
 2   ENCOUNTER    object
 3   CATEGORY     object
 4   CODE         object
 5   DESCRIPTION  object
 6   VALUE        object
 7   UNITS        object
 8   TYPE         object
dtypes: object(9)
memory usage: 284.6+ MB


Este conjunto de datos es considerablemente amplio, con **4.144.942 registros y 9 variables** que describen distintas interacciones clínicas de los pacientes. Contiene información sobre la fecha, tipo de encuentro, categoría médica, código y descripción de cada evento, así como los valores y unidades asociados. Representa un registro detallado de las actividades y mediciones realizadas en el contexto sanitario.

Dentro de este conjunto se identifica nuestra **variable objetivo 'target', que indica si el paciente cuenta con un registro de INR** y el valor correspondiente. Esta variable permite evaluar la presencia y magnitud de dicha medición, siendo la clave de este análisis.

## 2.2 Generación del csv

Se filtran los registros para incluir únicamente **pacientes vivos**, **medicación especifica de anticoagulantes** y aquellos pacientes con **controles de INR**, dado que el análisis se enfoca en esta medición. A continuación, se procede a **limpiar y depurar** los datos del subconjunto obtenido, asegurando su calidad y consistencia para el estudio.

##### **Pacientes difuntos**  
La columna DeathDate indica la fecha de fallecimiento; se eliminarán todos los registros donde este valor no sea nulo, conservando únicamente a los pacientes vivos para el análisis.

In [5]:
print("Cantidad de pacientes vivos:")
print(df_patients['DEATHDATE'].isna().sum())

print("Cantidad de pacientes difuntos:")
print(df_patients['DEATHDATE'].notna().sum())


Cantidad de pacientes vivos:
5000
Cantidad de pacientes difuntos:
693


Actualmente contamos con **5,000 pacientes vivos y 826 pacientes difuntos**. Para el análisis, conservaremos únicamente a los pacientes vivos, eliminando los registros correspondientes a los fallecidos.

In [6]:
df_patients = df_patients[df_patients['DEATHDATE'].isna()]

Para la variable **edad**, primero calculamos los años a partir de la fecha de nacimiento utilizando la diferencia con la fecha actual. No contamos con valores nulos en las fechas de nacimiento, asi que no tendremos problema en la transformación de los datos.

In [7]:
#convertimos la fecha de nacimiento en formato datetime para el calculo
df_patients['BIRTHDATE'] = pd.to_datetime(df_patients['BIRTHDATE'], errors='coerce')

df_patients['AGE'] = ((pd.Timestamp('today') - df_patients['BIRTHDATE']).dt.days / 365.25).astype(int)
df_patients['AGE'] = df_patients['AGE'].fillna(df_patients['AGE'].median())
df_patients = df_patients[['Id','AGE', 'GENDER']]

Una vez filtrados, de cada paciente solo nos interesan tres variables clave: edad, género e identificador, que serán utilizadas para los análisis posteriores.

In [8]:
df_observations = df_observations[['PATIENT','DESCRIPTION', 'VALUE']]
df_observations.head(5)

,PATIENT,DESCRIPTION,VALUE
0,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,Body Height,51.6
1,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,Pain severity - 0-10 verbal numeric rating [Sc...,3.0
2,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,Body Weight,3.2
3,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,Weight-for-length Per age and sex,1.4
4,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,Head Occipital-frontal circumference Percentile,35.2


### 2.2.1 Simplificación de las variables y pivotación

Las descripciones de las observaciones indican qué tipo de medición se ha realizado a cada paciente. Para tener una visión clara del contenido del dataset, extraeremos un listado único de los valores registrados en esta columna, lo que nos permitirá identificar y comprender todas las mediciones disponibles.

In [9]:
df_observations['DESCRIPTION'].unique()

array(['Body Height',
       'Pain severity - 0-10 verbal numeric rating [Score] - Reported',
       'Body Weight', 'Weight-for-length Per age and sex',
       'Head Occipital-frontal circumference Percentile',
       'Head Occipital-frontal circumference', 'Diastolic Blood Pressure',
       'Systolic Blood Pressure', 'Heart rate', 'Respiratory rate',
       'Leukocytes [#/volume] in Blood by Automated count',
       'Erythrocytes [#/volume] in Blood by Automated count',
       'Hemoglobin [Mass/volume] in Blood',
       'Hematocrit [Volume Fraction] of Blood by Automated count',
       'MCV [Entitic volume] by Automated count',
       'MCH [Entitic mass] by Automated count',
       'MCHC [Mass/volume] by Automated count',
       'Erythrocyte distribution width [Entitic volume] by Automated count',
       'Platelets [#/volume] in Blood by Automated count',
       'Platelet distribution width [Entitic volume] in Blood by Automated count',
       'Platelet mean volume [Entitic volume] in

Para simplificar el conjunto de variables, utilizamos los métodos auxiliares disponibles. En primer lugar, empleamos `cleaner_category_match` , que nos permite identificar cuántas variables categóricas existen dentro de cada categoría y cómo se emparejan con las cadenas de texto presentes en las observaciones. Este mismo módulo también facilita añadir o eliminar categorías según las necesidades del análisis, ofreciendo un control flexible sobre la organización del dataset.

Las categorías globales disponibles incluyen: vitals, hematology, chemistry coagulation, urinalysis, microbiology, infectious, immunology_allergy, oncology, ophthalmology_imaging, mental_health, social_determinants, demographics, substance_use, administrative y other.

Sin embargo, para este análisis nos centraremos únicamente en las categorías más relevantes: vitals, hematology, chemistry coagulation y un conjunto reducido de observaciones específicas relacionadas con el estado físico y conductual del paciente. Entre estas últimas se encuentran:

- Total score [DAST-10]: puntuación total del Drug Abuse Screening Test, un cuestionario que evalúa el riesgo o presencia de uso problemático de drogas.

- Total score [AUDIT-C]: puntuación del Alcohol Use Disorders Identification Test – Consumption, una medida breve que detecta consumo riesgoso de alcohol.

- PROMIS-10 Global Mental Health (GMH) score: indicador estandarizado que refleja el estado general de salud mental, incluyendo síntomas como ansiedad, depresión y bienestar emocional.

- PROMIS-10 Global Physical Health (GPH) score: medida que evalúa el estado general de salud física, considerando aspectos como dolor, fatiga, movilidad y percepción de salud.

Al quedarnos solo con estas categorías y métricas específicas, garantizamos que el análisis se centre en las variables más relevantes y que la información sea manejable y coherente para los objetivos del estudio.

In [10]:

# Diagnostico de las categorias existentes
diagnose_category_match(df_observations, 'urinalysis', column='DESCRIPTION')

# Eliminar valores de categorias concretas
df_observations = drop_categories(df_observations, ['urinalysis','microbiology','infectious','immunology_allergy','oncology','ophthalmology_imaging','mental_health','social_determinants','demographics','substance_use','administrative','other'], column='DESCRIPTION', fuzzy=True)

#df_observations['DESCRIPTION'].unique()


=== Diagnosis for category: urinalysis ===
Variables in category: 28
Unique values in df['DESCRIPTION']: 300

Exact matches found: 27
Examples: ['Microalbumin/Creatinine [Mass Ratio] in Urine', 'WBCs', 'Bilirubin.total [Presence] in Urine by Test strip']

Partial matches found: 34
Examples: ['Microalbumin/Creatinine [Mass Ratio] in Urine', 'WBCs', 'Bilirubin.total [Presence] in Urine by Test strip']
Dropping 2228151 rows from categories: ['urinalysis', 'microbiology', 'infectious', 'immunology_allergy', 'oncology', 'ophthalmology_imaging', 'mental_health', 'social_determinants', 'demographics', 'substance_use', 'administrative', 'other']
Sample of dropped values: ['Pain severity - 0-10 verbal numeric rating [Score] - Reported', 'Weight-for-length Per age and sex', 'Head Occipital-frontal circumference Percentile', 'Head Occipital-frontal circumference', 'Body mass index (BMI) [Percentile] Per age and sex']


A continuación, dado que muchas variables poseen nombres extensos y poco intuitivos para utilizarlos como columnas del dataset, recurrimos al segundo método auxiliar: `cleaner_shorter_categories`. Este recurso nos permite acortar y estandarizar dichas cadenas, generando nombres más claros y manejables sin perder información relevante.

In [11]:
df_observations = shorten_variable_names(
    df_observations, 
    column='DESCRIPTION', 
    inplace=True  # Replace the column
)

df_observations['DESCRIPTION'].unique()

Shortened 592951/1916791 variable names (30.9%)


array(['Height', 'Weight', 'DBP', 'SBP', 'Heart_Rate', 'Respiratory_Rate',
       'WBC', 'RBC', 'Hemoglobin', 'Hematocrit', 'MCV', 'MCH', 'MCHC',
       'RDW', 'Platelets', 'PDW', 'MPV', 'Smoking_Status', 'Temperature',
       'BMI', 'O2_Saturation', 'Stress_Level', 'Cholesterol_Total',
       'Triglycerides', 'LDL', 'HDL', 'Glucose', 'BUN', 'Creatinine',
       'Calcium', 'Sodium', 'Potassium', 'Chloride', 'CO2', 'eGFR',
       'Protein', 'Albumin', 'Globulin', 'Bilirubin_Total', 'ALP', 'ALT',
       'AST', 'HbA1c', 'AUDIT_C', 'INR', 'PT', 'RDW_Ratio', 'Magnesium',
       'aPTT', 'NT_proBNP', 'Troponin_I', 'Neutrophils_Pct',
       'Lymphocytes_Pct', 'Monocytes_Pct', 'Eosinophils_Pct',
       'Basophils_Pct', 'Neutrophils_Abs', 'Lymphocytes_Abs',
       'Monocytes_Abs', 'Eosinophils_Abs', 'Basophils_Abs', 'pH_Arterial',
       'pCO2_Arterial', 'pO2_Arterial', 'HCO3_Arterial', 'FiO2',
       'D_Dimer', 'Ferritin', 'LDH', 'CK', 'CRP', 'Procalcitonin', 'IL6',
       'FEV1_FVC_Ratio', 'MM

Finalmente, realizaremos **un pivotado de los datos para transformar las entradas de ‘DESCRIPTION’ y ‘VALUE’ en columnas independientes**, cada una con sus valores correspondientes. Esto permite reorganizar la información en un formato más estructurado y adecuado para el análisis.

In [12]:
df_observations_pivot = df_observations.pivot_table(
    index='PATIENT',
    columns='DESCRIPTION',
    values='VALUE',
    aggfunc='first'
).reset_index()

df_observations_pivot = df_observations_pivot.rename_axis(None, axis=1)

df_observations_pivot.head()

,PATIENT,ACT,ALP,ALT,ALT_Elevated,AST,AST_Elevated,AUDIT_C,Abdominal_Exam,Albumin,...,WBC,Weight,aPTT,eGFR,pCO2_Arterial,pCO2_Venous,pH_Arterial,pH_Venous,pO2_Arterial,pO2_Venous
0,000d67c2-0a4a-9a55-b263-c48c288deeee,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,7.6,47.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000e0ae6-db59-4fe0-b099-13be72a72a42,115.2,71.3,17.9,NaN,37.5,NaN,2.0,NaN,4.8,...,7.2,85.5,31.6,61.2,37.7,44.0,7.2,7.3,98.9,45.0
2,00164874-b655-b577-f4a0-e170782b0690,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,8.9,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0018d8bf-328d-3cb7-5d7d-c363f4c5da1e,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,9.9,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00436b14-83c0-df1e-8f99-c8b7ba28072f,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,8.4,98.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A continuación, uniremos la información de los pacientes con sus respectivas observaciones utilizando el identificador único (Id) de cada paciente. Esta combinación nos permitirá consolidar los datos en un solo dataset, facilitando el análisis y asegurando que toda la información relevante de cada paciente esté centralizada.

In [13]:
# - df_patients (izquierda) tiene la columna clave 'Id'
# - df_observations (derecha) tiene la columna clave 'PATIENT'

df_patients_observations = pd.merge(
    df_patients,        # DataFrame de la izquierda
    df_observations_pivot,  # DataFrame de la derecha
    left_on='Id',           # <--- Columna clave en el DataFrame de la izquierda (df_patients)
    right_on='PATIENT',     # <--- Columna clave en el DataFrame de la derecha (df_observations)
    how='left'              # Tipo de unión (mantiene todos los pacientes del df_patients)
)

#Eliminamos la columna PATIENT para no tener duplicados en el identificador
columnas_a_eliminar = ['PATIENT']
df_patients_observations = df_patients_observations.drop(columns=columnas_a_eliminar)
df_patients_observations.head()

,Id,AGE,GENDER,ACT,ALP,ALT,ALT_Elevated,AST,AST_Elevated,AUDIT_C,...,WBC,Weight,aPTT,eGFR,pCO2_Arterial,pCO2_Venous,pH_Arterial,pH_Venous,pO2_Arterial,pO2_Venous
0,ae1acd06-3eb7-c5c5-738f-c9895ef4150d,9,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.7,3.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a48ad472-6249-d9df-9352-6d04658dc834,30,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.3,78.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6875adfb-1f3e-1e01-2c4d-1dc19a5c04bc,7,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,56689f99-ca4d-67eb-12d6-5b0f0dc45b59,11,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.7,8.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9169a441-7be6-032e-690b-0c09c868c9e7,23,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.2,55.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### **Pacientes con INR**  

Dado que este análisis **se centrará en pacientes con medición de INR**, eliminaremos todas las filas correspondientes a aquellos que no cuentan con esta observación. Tras este filtrado, nos quedamos con un total de 434 pacientes y 103 columnas, sobre los cuales realizaremos el análisis detallado.

In [14]:
df_patients_observations_INR = df_patients_observations[
    (df_patients_observations['INR'] != 0) & 
    (df_patients_observations['INR'].notna())
]

df_patients_observations_INR.head()

,Id,AGE,GENDER,ACT,ALP,ALT,ALT_Elevated,AST,AST_Elevated,AUDIT_C,...,WBC,Weight,aPTT,eGFR,pCO2_Arterial,pCO2_Venous,pH_Arterial,pH_Venous,pO2_Arterial,pO2_Venous
19,b195bb80-5573-fad3-6bda-1b90a5b06441,55,F,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,9.4,79.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,ca4c7b74-6913-7b3f-31a9-716de55fcb92,61,M,NaN,91.1,9.4,NaN,1.3,NaN,3.0,...,5.1,74.6,28.5,7.9,NaN,NaN,NaN,NaN,NaN,NaN
24,f1e74d04-5033-b9d5-34c2-9d65401e90a9,75,M,127.4,23.0,34.9,NaN,25.6,NaN,1.0,...,8.6,92.3,38.4,75.7,38.1,46.7,7.2,7.3,99.3,43.1
47,ef4ee01b-b0a7-d6ea-b613-0d78c893e33c,98,F,NaN,66.3,51.4,NaN,34.8,NaN,2.0,...,3.8,74.4,NaN,86.4,NaN,NaN,NaN,NaN,NaN,NaN
57,16342f9e-307c-9c4a-d4b0-866cac94d751,44,M,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,9.8,104.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### **Medicación especifica de anticoagulantes** 

A continuación, identificaremos a los pacientes que toman anticoagulantes, filtrando únicamente las medicaciones que sean anticoagulantes.

In [15]:
df_medications['DESCRIPTION'].unique()

array(['Penicillin V Potassium 250 MG Oral Tablet',
       'Acetaminophen 160 MG Chewable Tablet',
       'Ibuprofen 100 MG Oral Tablet', 'Loratadine 5 MG Chewable Tablet',
       'NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector',
       'Acetaminophen 300 MG / Hydrocodone Bitartrate 5 MG Oral Tablet',
       'tramadol hydrochloride 50 MG Oral Tablet',
       'sodium fluoride 0.0272 MG/MG Oral Gel',
       'Amoxicillin 500 MG Oral Tablet', 'Cefuroxime 250 MG Oral Tablet',
       'amoxicillin 500 MG / clavulanate 125 MG Oral Tablet',
       'Levora 0.15/30 28 Day Pack', 'Natazia 28 Day Pack',
       'Seasonique 91 Day Pack', 'ferrous sulfate 325 MG Oral Tablet',
       'Vitamin B12 5 MG/ML Injectable Solution',
       'Simvastatin 10 MG Oral Tablet',
       'Meperidine Hydrochloride 50 MG Oral Tablet',
       'Ibuprofen 200 MG Oral Tablet',
       'Acetaminophen 325 MG / Oxycodone Hydrochloride 5 MG Oral Tablet',
       'Hydrochlorothiazide 25 MG Oral Tablet',
       'lisinopril 10 M

De manera similar a lo que hicimos con las observaciones, donde aplicamos métodos para eliminar categorías innecesarias y simplificar los nombres, realizaremos un proceso equivalente con los datos de medicamentos. Para ello, utilizaremos métodos específicos como `categorize_meds` y `drop_meds_by_category`. Para la simplificacion crearemos un

In [16]:
# Categorizar medicamentos
df_medications = categorize_meds(df_medications)
# Ahora sí se puede filtrar
df_medications = drop_meds_by_category(df_medications, [
    'antibiotic', 'analgesic','antihypertensive','antidiabetic','hormone','statin','inhaler','immunotherapy', 'other'
])


Dropping 262039 medications from categories: ['antibiotic', 'analgesic', 'antihypertensive', 'antidiabetic', 'hormone', 'statin', 'inhaler', 'immunotherapy', 'other']
Sample dropped: ['Penicillin V Potassium 250 MG Oral Tablet', 'Acetaminophen 160 MG Chewable Tablet', 'Ibuprofen 100 MG Oral Tablet', 'Loratadine 5 MG Chewable Tablet', 'NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Injector']


In [17]:
df_medications['DESCRIPTION'].unique()

array(['0.4 ML Enoxaparin sodium 100 MG/ML Prefilled Syringe',
       '1 ML Enoxaparin sodium 150 MG/ML Prefilled Syringe', 'Warfarin',
       '1 ML heparin sodium  porcine 5000 UNT/ML Injection',
       'Warfarin Sodium 5 MG Oral Tablet',
       'heparin sodium  porcine 100 UNT/ML Injectable Solution',
       'warfarin sodium 5 MG Oral Tablet', 'Apixaban',
       'heparin sodium  porcine 1000 UNT/ML Injectable Solution',
       'enoxaparin sodium 100 MG/ML Injectable Solution', 'Rivaroxaban'],
      dtype=object)

In [18]:
df_medications = shorten_variable_names_med(
    df_medications, 
    column='DESCRIPTION', 
    inplace=True  # Replace the column
)

df_medications['DESCRIPTION'].unique()

Shortened 0/3915 variable names (0.0%)


array(['Enoxaparin_100', 'Enoxaparin_150', nan, 'Heparin_5000',
       'Warfarin_5', 'Heparin_100', 'Heparin_1000'], dtype=object)

In [19]:
df_medications = df_medications[['PATIENT','DESCRIPTION']]

# - df_patients (izquierda) tiene la columna clave 'Id'
# - df_observations (derecha) tiene la columna clave 'PATIENT'

df_patients_observations_medications_INR = pd.merge(
    df_patients_observations_INR,        # DataFrame de la izquierda
    df_medications,  # DataFrame de la derecha
    left_on='Id',           # <--- Columna clave en el DataFrame de la izquierda (df_patients)
    right_on='PATIENT',     # <--- Columna clave en el DataFrame de la derecha (df_observations)
    how='left'              # Tipo de unión (mantiene todos los pacientes del df_patients)
)

#Eliminamos la columna PATIENT para no tener duplicados en el identificador
columnas_a_eliminar = ['PATIENT']
df_patients_observations_medications_INR = df_patients_observations_medications_INR.drop(columns=columnas_a_eliminar)
df_patients_observations_medications_INR = df_patients_observations_medications_INR.rename(columns={'DESCRIPTION': 'Medication'})
df_patients_observations_medications_INR.to_csv('data/df_patients_observations_medications_INR_transformed.csv')
df_patients_observations_medications_INR.to_csv('data/df_patients_observations_medications_INR.csv')
df_patients_observations_medications_INR.head()

OSError: Cannot save file into a non-existent directory: 'data'

Se mantendran únicamente con los medicamentos asociados a anticoagulantes, descartando el resto. Estos datos serán útiles para análisis posteriores, por ejemplo, para identificar qué medicaciones son más frecuentes en pacientes con coagulación fuera de rango y para estudiar posibles relaciones entre tratamientos y resultados clínicos.

Por el momento, sin embargo, no consideraremos las medicaciones en el análisis principal del modelo predictivo, aunque su preparación nos permitirá integrarlas fácilmente en futuros estudios.

## 2.3 Limpieza de datos

Los modelos de aprendizaje automático tienden a mejorar su precisión cuando se entrenan con grandes volúmenes de datos; no obstante, el tamaño de este conjunto podría no ser suficiente para alcanzar un rendimiento óptimo, por lo que evaluaremos si este es el caso.

En las siguientes secciones nos enfocaremos en limpiar y preparar los datos para garantizar que el modelo trabaje con la información más confiable posible. Esto implica corregir errores de etiquetado o tipográficos, manejar valores faltantes y ajustar los tipos de datos según sea necesario.

Además, muchos modelos requieren que los datos estén correctamente escalados y no contengan valores nulos para poder converger de manera efectiva. Dado que nuestro objetivo es que el modelo identifique patrones que no son evidentes a simple vista, aprovecharemos la información que sí podemos depurar para facilitar su aprendizaje y optimizar el proceso de entrenamiento.

### 2.3.1 Valores faltantes y data types

In [ ]:
info_complete = pd.DataFrame({
    'Dtype': df_patients_observations_medications_INR.dtypes,
    'Non-Null Count': df_patients_observations_medications_INR.notnull().sum()
})

# Mostrar todo
pd.set_option('display.max_rows', None)  # permite mostrar todas las filas
print(info_complete)

El conjunto de datos presenta una composición muy desequilibrada: aunque contiene 2283 registros y un amplio abanico de 104 variables clínicas, la disponibilidad de información varía drásticamente entre ellas. 
Por un lado, las *mediciones básicas; como los signos vitales, los parámetros antropométricos y la mayoría de elementos del hemograma están prácticamente completas*. Esto ofrece una base sólida para análisis poblacionales generales y modelos que se apoyen en información fundamental del estado de salud.

En contraste, la mayoría de las *pruebas de laboratorio más especializadas apenas cuentan con datos suficientes*. Marcadores inflamatorios, enzimas hepáticas, biomarcadores cardiacos, gases arteriales y venosos, hormonas tiroideas o perfiles de hierro aparecen registrados solo en una fracción muy pequeña de los pacientes. Esta escasez limita su uso directo en análisis estadísticos o modelos predictivos, a menos que se recurra a métodos avanzados de imputación o se restrinja el estudio a subgrupos específicos.

En conjunto, el **dataset parece reflejar la dinámica típica de un entorno clínico real, donde las pruebas complejas solo se solicitan a determinados pacientes**, lo que introduce un fuerte sesgo en la disponibilidad de estos datos. En consecuencia, el conjunto es adecuado para investigaciones amplias basadas en variables generales, pero presenta claras limitaciones para estudios que dependan de biomarcadores avanzados o análisis integrales de laboratorio.

En cuanto a los tipos de datos, observamos que muchos campos numéricos, tanto enteros como decimales, están siendo interpretados incorrectamente como variables categóricas. En realidad, solo los campos Smoking_Status, Stress_Level, Medication y Gender contienen información categórica relevante que explica comportamientos o condiciones del paciente. Por ello, procederemos a transformar los datos, asegurando que cada variable tenga el tipo adecuado para el análisis y el entrenamiento de los modelos, lo que permitirá un procesamiento más preciso y coherente.

In [ ]:
df_patients_observations_medications_INR = df_patients_observations_medications_INR.apply(lambda x: x.astype(str).str.strip())  # quitar espacios y convertir todo a string

exclude_cols = ["Id","Smoking_Status", "Stress_Level","Medication","GENDER"]

cols_to_convert = [col for col in df_patients_observations_medications_INR.columns if col not in exclude_cols]

df_patients_observations_medications_INR[cols_to_convert] = df_patients_observations_medications_INR[cols_to_convert].apply(pd.to_numeric, errors="coerce").astype("float64")

df_patients_observations_medications_INR.info()

Las variables categóricas, como el estado de tabaquismo, deben transformarse en variables binarias o dummies antes de incluirlas en la regresión, para que el modelo interprete correctamente la información sin asumir un orden numérico inexistente.

Para este caso, las variables categoricas se transformaran en *variables binarias en vez de utilizar la tecnica One-hot encoding* para evitar la sobrecreación de columnas, y la problematica de dimensionalidad posterior.

**Variable Smoking_Status**

Se agruparan a quienes fuman actualmente y a quienes fumaron en el pasado dentro de la misma categoría binaria (1) dado que el cuerpo conserva huellas duraderas de la exposición al tabaco. Aunque un exfumador ya no consuma cigarrillos, su historia de consumo sigue influyendo en procesos biológicos, riesgos cardiovasculares y en la respuesta a ciertos tratamientos. En cambio, quienes nunca han fumado parten de un perfil fisiológico distinto.

Al convertir la variable en 0 = nunca fumó y 1 = fumador o exfumador, estamos simplificando pero puediendo distinguir entre quienes han estado expuestos al efecto del tabaco y quienes no.

In [ ]:
# Diccionario para transformar a binario
smoking_binary = {
    'Ex-smoker (finding)': 1,
    'Smokes tobacco daily (finding)': 1,
    'Never smoked tobacco (finding)': 0
}

# Aplicar la transformación
df_patients_observations_medications_INR['Smoking_Status'] = (
    df_patients_observations_medications_INR['Smoking_Status'].map(smoking_binary)
)

**Variable Stress_Level**

La conversión del nivel de estrés en una variable binaria tiene como objetivo diferenciar a quienes presentan ningún estrés de aquellos cuyo nivel puede afectar de manera significativa procesos fisiológicos, conductuales y la respuesta a tratamientos. Al codificar 0 = sin estrés y 1 = estrés bajo/alto, logramos capturar la diferencia que resulta realmente relevante desde el punto de vista clínico y analítico.

Las respuestas “Somewhat”, “Quite a bit” y “Very much” se considerarán como indicativos de estrés, aunque puedan reflejar distintos grados, mientras que “Not at all” representa un estado sin impacto significativo sobre la salud.

In [ ]:
# Diccionario para binarizar Stress_Level
stress_binary = {
    'Not at all': 0,
    'A little bit': 1,
    'Somewhat': 1,
    'Quite a bit': 1,
    'Very much': 1,
    'I choose not to answer this question': 0
}

# Aplicar la transformación
df_patients_observations_medications_INR['Stress_Level'] = (
    df_patients_observations_medications_INR['Stress_Level'].map(stress_binary)
)


### 2.3.2 Duplicados

In [ ]:
df_patients_observations_medications_INR.duplicated().sum()

El método duplicated() devuelve False para las filas que no están duplicadas. Luego, al usar sum() sobre los resultados, solo se contabilizan los valores True. Dado que obtenemos 1740 valores True en 2283 filas, podemos concluir que **existen filas duplicadas en el dataset**. Eliminamos los registros duplicados.

In [ ]:
df_patients_observations_medications_INR = df_patients_observations_medications_INR.drop_duplicates()

### 2.3.3 Eliminación de variables redundantes o irrelevantes

Para simplificar nuestro análisis y centrarnos en las variables más relevantes para la predicción del INR, eliminamos aquellas columnas que aportan información irrelevante o redundante. Esto incluye mediciones físicas generales, marcadores de laboratorio no relacionados directamente con la coagulación, y algunas columnas duplicadas o categóricas que no utilizaremos en esta etapa. Con esto, reducimos el ruido en los datos y facilitamos que el modelo aprenda patrones significativos de manera más eficiente.

In [ ]:
df_patients_observations_medications_INR
cols_to_drop = ["Height", "Weight","Mean_BP","Abdominal_Exam","RDW_Ratio","eGFR","Globulin","HbA1c","MCV","MCH","MCHC","RDW","MPV","Stress_Test","Basophils_Pct","Globulin","Eosinophils_Pct","Eosinophils_Abs","D_Dimer","Procalcitonin","Neutrophils_Abs","Neutrophils_Pct","Lymphocytes_Abs","Lymphocytes_Pct","Monocytes_Abs","Monocytes_Pct","Basophils_Abs","Cholesterol_Total","Basophils_Pct","Medication"]

df_patients_observations_medications_INR.drop(cols_to_drop, axis=1, inplace=True)

### 2.3.4 Rangos y estadística básica

In [ ]:
df_patients_observations_medications_INR.describe(include = np.number).round(2)

In [ ]:
# Check the stats of categorical features
df_patients_observations_medications_INR.describe(include='object')

El analisis anterior muestra una estructura irregular ya que algunas columnas cuentan con una cantidad aceptable de registros, mientras que otras apenas tienen observaciones, lo que las vuelve poco aprovechables. 
Se identifican además variables completamente estáticas, sin variación (std 0.0), y otras con valores extremos.

Para tratar estos datos estableceremos lo siguiente:

Criterios entorno a la **eliminación de columnas**:

1. Cobertura muy baja: columnas con muy pocos registros (<5% de los pacientes)
2. Pruebas especializadas: Hay biomarcadores o pruebas que solo se realizaron en subgrupos y no aportan información general.

Ejemplos de columnas a eliminar: ALT_Elevated, AST_Elevated, Bilirubin_Elevated, Anion_Gap, CD4_Count

In [ ]:
cols_to_drop = ["ALT_Elevated", "AST_Elevated", "Anion_Gap","Bilirubin_Elevated","CD4_Count","Capillary_Refill", "FEV1_FVC_Ratio","FiO2","Free_T4","Iron","Iron_Saturation","Lactate","MMSE","NYHA_Functional","NYHA_Objective","TIBC"]
df_patients_observations_medications_INR.drop(cols_to_drop, axis=1, inplace=True)

Por otro lado, la **imputación de valores faltantes** solo tiene sentido en columnas que sean relativamente completas y clínicamente relevantes. Los criterios para aplicar la imputación serian:

1. Cobertura moderada: al menos 40–70% de los registros disponibles.
2. Relevancia clínica: variables importantes para análisis y el modelo predictivo.

Ejemplos de columnas a aplicar la imputación: Temperature, BMI.

Las variables continuas se imputaran con la mediana y las variables categoricas con la moda.


In [ ]:
numerical = df_patients_observations_medications_INR.select_dtypes(include=['int64','float64']).columns.tolist()

for col in numerical:
    if df_patients_observations_medications_INR[col].isnull().any():
        # Use the built-in round() function on the float result of .mean()
        mean_value = round(df_patients_observations_medications_INR[col].mean(), 2) 
        df_patients_observations_medications_INR[col].fillna(mean_value, inplace=True)

### 2.3.5 Valores únicos

El método `describe()` utilizado previamente muestra los valores únicos para las variables categóricas, ahora comprobamos los valores únicos de las características numéricas

In [ ]:
for column in numerical:
  print(f"{column} has {df_patients_observations_medications_INR[column].nunique()} unique values.")

Los resultados que obtenermos son coherentes, no se necesita limpiar más los datos.

## 2.4 Feature importance y analisis de la variable 'target'

#### 2.4.1 Análisis de dependencia y asociación

Ahora procederemos a realizar un análisis de dependencia y asociación entre las variables del dataset y la variable objetivo, INR. El objetivo de este paso es identificar cuáles características tienen un efecto relevante sobre INR y cuáles podrían aportar información útil para la predicción, antes de pasar a modelos más complejos.

Para ello, comenzamos con **ANOVA (Análisis de Varianza)**. Esta técnica permite evaluar si las diferencias observadas en la variable objetivo entre distintos grupos de una variable explicativa son estadísticamente significativas. En otras palabras, ANOVA nos indica si una variable puede influir de manera importante en INR al comparar las medias de INR entre sus distintos niveles o rangos. Es especialmente útil para detectar relaciones lineales o aditivas y nos ayuda a priorizar variables que muestran un efecto claro sobre la variable objetivo, sentando las bases para análisis posteriores más sofisticados como la información mutua o la correlación.

In [ ]:
target_col = 'INR'

# Crear la columna categórica INR_Group
bins = [df_patients_observations_medications_INR[target_col].min() - 1, 2.0, 3.0 + 1e-6, df_patients_observations_medications_INR[target_col].max() + 1]
df_patients_observations_medications_INR['INR_Group'] = pd.cut(
    df_patients_observations_medications_INR[target_col], 
    bins=bins, 
    labels=['Bajo (<2.0)', 'Normal (2.0-3.0)', 'Alto (>3.0)'], 
    right=False, 
    include_lowest=True
).astype('category')

In [ ]:
anova_results = {}

for col in df_patients_observations_medications_INR.columns:
    if col not in ["INR", "INR_Group"] and pd.api.types.is_numeric_dtype(df_patients_observations_medications_INR[col]):
        groups = [
            df_patients_observations_medications_INR[col][df_patients_observations_medications_INR["INR_Group"] == g]
            for g in df_patients_observations_medications_INR["INR_Group"].cat.categories
        ]
        
        # Comprobar que cada grupo tenga más de un valor único
        if all(len(g.unique()) > 1 for g in groups):
            f_stat, p_val = stats.f_oneway(*groups)
            anova_results[col] = {"F": f_stat, "p_value": p_val}
        else:
            print(f"Skipping {col}: some group has constant values")

anova_df = pd.DataFrame(anova_results).T
anova_df.sort_values("p_value").head()


El análisis de varianza (ANOVA) se utilizó para evaluar si existían diferencias significativas en diversas variables clínicas y demográficas entre los distintos grupos de INR (Bajo (<2.0), Normal (2.0-3.0) y Alto (>3.0)). El estadístico F refleja la relación entre la variabilidad entre grupos y la variabilidad dentro de los grupos; un valor alto de F indica que las medias de los grupos son considerablemente diferentes. Por su parte, el valor p (p_value) representa la probabilidad de obtener un F al menos tan extremo bajo la hipótesis nula de que no existen diferencias entre grupos; valores de p menores a 0.05 se consideran evidencia de diferencias significativas.

Al examinar las variables de forma individual, se observaron los siguientes hallazgos:

- pO2 arterial: presentó un F de 23.02 y un p muy bajo (≈ 2.5e-10), indicando diferencias altamente significativas entre los grupos de INR. Esto sugiere que la presión arterial de oxígeno varía de manera clara según si el INR es bajo, normal o alto.

- Saturación de oxígeno (O2_Saturation): con un F de 7.42 y p ≈ 0.00066, también mostró diferencias significativas, lo que implica que la saturación de oxígeno se ve influida por el nivel de INR.

- Edad (AGE): presentó un F de 6.59 y p ≈ 0.00149, señalando diferencias significativas en la edad promedio entre los grupos de INR.

- Albúmina: con un F de 3.02 y p ≈ 0.0495, mostró diferencias marginalmente significativas, sugiriendo que la albúmina podría variar entre los grupos, aunque con menor intensidad que las variables anteriores.

- Proteína total (Protein): presentó un F de 1.51 y p ≈ 0.223, lo que indica que no existen diferencias significativas entre los grupos de INR; esta variable no parece estar relacionada con el nivel de INR.

En resumen, los resultados muestran que algunas variables clínicas, como la presión arterial de oxígeno, la saturación de oxígeno y la edad, difieren significativamente según el grupo de INR, mientras que otras, como la proteína total, no presentan diferencias evidentes. La albúmina podría estar asociada de forma débil, pero su efecto es menos marcado.

#### 2.4.2 Análisis de pdredicción y relevancia

A continuación, se procederá a realizar un análisis de predicción y relevancia con el objetivo de identificar qué variables aportan información significativa para explicar y predecir la variabilidad del INR. Para ello, se empleará **información mutua (MI)**, una medida que captura la dependencia entre variables sin asumir linealidad, lo que permite detectar relaciones complejas o no evidentes entre los factores clínicos y la variable objetivo.

Se ha elegido un umbral de **MI de 0.15**, lo suficientemente bajo para no descartar variables con señal débil pero relevante, y al mismo tiempo alto para evitar incluir características que aporten información mínima o ruido. Aplicando este criterio, se identificaron como variables más relevantes: Troponin_I, aPTT, AST, Magnesium, pO2_Arterial, ALP, Bilirubin_Total

Con base en estos resultados, se procederá a reducir el dataset a estas columnas (se incluirá Id y GENDER por defecto), manteniendo solo las variables con información significativa para la predicción de INR, lo que facilitará análisis posteriores y modelado más eficiente. Sin embargo, se **excluirá PT del análisis final**, ya que esta medida representa directamente el INR y no aporta información externa adicional sobre los factores que afectan la coagulación. Nuestro interés se centra en identificar variables clínicas y bioquímicas que influyan en la coagulación de manera más independiente y que puedan aportar valor explicativo o predictivo más allá de la medida básica de INR.

Para el análisis de información mutua, se creó una columna categórica a partir del INR en lugar de usarlo como continuo. Esto permite identificar más fácilmente qué variables están asociadas a cambios relevantes en los rangos de INR (“Bajo”, “Normal”, “Alto”) y evita que pequeñas variaciones numéricas generen ruido, haciendo los resultados más interpretables y útiles.

In [ ]:

# Variable objetivo para MI
target_col = 'INR_Group'
threshold = 0.15
non_numeric_cols_to_keep = ['Id', 'GENDER']

# Identificar variables numéricas y categóricas
df_features = df_patients_observations_medications_INR.drop(columns=[target_col, 'Id','PT'])

numeric_cols = df_features.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_features.select_dtypes(include=['object', 'category']).columns.tolist()

# Convertir categóricas a numericas
df_encoded = df_features.copy()
for col in categorical_cols:
    df_encoded[col] = df_encoded[col].astype('category').cat.codes

# Indicar qué columnas son discretas (todas las categóricas codificadas)
discrete_features = [col in categorical_cols for col in df_encoded.columns]

# Calcular MI
X = df_encoded
y = df_patients_observations_medications_INR[target_col]
mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features, random_state=42)
mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Filtrar variables que superan el umbral
selected_mi = mi_series[mi_series >= threshold]
print(selected_mi)
# Gráfico de Información Mutua
plt.figure(figsize=(14, 16))
sns.set_style("whitegrid")
plot_data = mi_series.reindex(mi_series.sort_values(ascending=False).index)
colors = ['darkred' if v >= threshold else 'lightgray' for v in plot_data.values]

plt.barh(plot_data.index, plot_data.values, color=colors)
plt.axvline(threshold, color='green', linestyle='--', linewidth=1.5, label=f'Umbral MI ({threshold})')
plt.title(f'Importancia de las Variables según Información Mutua con {target_col}', fontsize=16)
plt.xlabel('Información Mutua', fontsize=14)
plt.ylabel('Variable', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
columns_to_keep = ['pO2_Arterial','O2_Saturation','AGE','Albumin','Troponin_I','aPTT','AST','Magnesium','ALP','Bilirubin_Total','INR']
#df_patients_observations_medications_INR = df_patients_observations_medications_INR[columns_to_keep]


#### 2.4.3 Correlación

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Numeric columns without target
numeric_cols = [
    col for col in df_patients_observations_medications_INR.select_dtypes(include=np.number).columns
    if col not in ['INR', 'PT']
]

target_col = 'INR'

# Pearson correlation matrix
corr_matrix = df_patients_observations_medications_INR[numeric_cols + [target_col]].corr(method='pearson')

# Correlation of all variables with INR
corr_with_inr = corr_matrix[target_col].drop(target_col).abs().sort_values(ascending=False)

# Select the top 25 columns
top_n = 25
top_cols = corr_with_inr.head(top_n).index.tolist()

# Include INR itself in the heatmap
cols_to_plot = top_cols + [target_col]

# Plot heatmap
plt.figure(figsize=(20, 16))  # Ampliamos el tamaño del gráfico
sns.set(font_scale=1.2)

sns.heatmap(
    corr_matrix.loc[cols_to_plot, cols_to_plot],
    annot=True,
    fmt=".2f",  # Redondea los floats a 2 decimales
    linewidths=.5,
    cmap="Blues"
)

plt.title(f'Top {top_n} Variables Most Correlated with INR', fontsize=18)
plt.show()


📜 Conclusión Narrativa Definitiva del Análisis de CorrelaciónEl análisis exhaustivo de esta matriz de correlación de Pearson, que abarca 25 variables clínicas, bioquímicas y demográficas, nos permite llegar a una conclusión clara y, quizás, sorprendente sobre el comportamiento del INR (International Normalized Ratio) en esta población.📉 El INR: Un Enigma LinealA pesar de incluir parámetros altamente relevantes para el metabolismo y la función hepática (como la Bilirrubina, ALT, Albúmina y Plaquetas), la conclusión principal es que el INR opera de forma linealmente independiente de todo el conjunto de variables.Al recorrer la fila del INR, el patrón es inconfundible: los coeficientes de correlación ($r$) están todos concentrados muy cerca del cero. Incluso las variables que tienen la relación más fuerte, la Bilirubin_Total y la pO2_Arterial ($r = -0.19$ en ambos casos), apenas demuestran una conexión lineal. Un valor de $-0.19$ se clasifica como una correlación débil; en términos prácticos, significa que la variación en estas variables explica una fracción insignificante de la variación en el INR.Getty ImagesExploreLos Múltiples Ceros: La mayoría de las variables, incluyendo marcadores fundamentales para la salud hepática (como la ALT) o esenciales para la coagulación (como el recuento de Plaquetas), muestran una correlación lineal virtualmente nula, con $r$ muy cercanos a $0$. Esto es un fuerte indicativo de que la causa real de la fluctuación del INR no reside en la variación de estos parámetros.🔗 Las Relaciones Biológicas FielesMientras que el INR se aísla de forma lineal, la matriz de correlación confirma varias relaciones biológicas y fisiológicas esperadas entre las otras variables:Sinergia de Oxígeno: Se observó una correlación positiva moderada y coherente de $0.50$ entre la pO2_Arterial y la O2_Saturation, validando que la presión de oxígeno en la sangre está directamente ligada a la cantidad de hemoglobina que transporta ese oxígeno.Presión Arterial: El vínculo entre la Presión Sistólica (SBP) y la Presión Diastólica (DBP) se confirmó con un robusto $\mathbf{0.50}$, lo cual es de esperar, ya que ambas son medidas de la misma fuerza cardiovascular.Vínculos Metabólicos: También se identificó un nexo moderado entre la Glucosa y los Triglycerides ($\mathbf{0.46}$), una relación común en pacientes con disfunción metabólica o riesgo cardiovascular.💡 Veredicto FinalEl análisis estadístico es definitivo: el INR no puede ser predicho de manera lineal y confiable utilizando estas 25 variables.La explicación más probable es que la variación del INR en esta cohorte está dominada por factores externos no incluidos en el modelo de correlación simple. Estos factores son generalmente el uso y la dosis de medicamentos anticoagulantes (como la Warfarina), o la presencia de una enfermedad subyacente severa (como cirrosis avanzada) cuyo impacto complejo no se refleja linealmente en estos marcadores de laboratorio.En conclusión, para comprender el INR, debemos buscar más allá de las correlaciones lineales simples y considerar factores farmacológicos o modelos de predicción multivariante más sofisticados.

Tras completar los tres análisis —ANOVA para evaluar dependencia, Información Mutua para medir relevancia predictiva y Pearson para identificar relaciones lineales— fue posible construir una visión conjunta y más equilibrada de qué variables realmente aportan información útil sobre el comportamiento del INR. En lugar de basarnos en un único criterio, se decidió priorizar aquellas variables que destacaban en más de una técnica, que tenían un respaldo clínico sólido o que contribuían a explicar el INR sin generar redundancia. Con este enfoque más integrador y prudente, y dado el límite máximo de quince variables, se obtuvo una selección final coherente tanto desde el punto de vista estadístico como desde el clínico.

En este conjunto reducido se incorporan, por un lado, marcadores que ANOVA identifica como fuertemente vinculados al INR, como LDH, pO₂_Arterial, Hemoglobin, AGE e IL6, todos ellos asociados a cambios significativos en la coagulación. Por otro lado, se incluyen variables que, aun sin mostrar relaciones lineales claras, revelan patrones relevantes mediante Información Mutua, como Glucose, HCO₃_Arterial, AST, Troponin_I, Protein, Albumin, ALT y O₂_Saturation, capaces de capturar dependencias más complejas. Finalmente, se añaden variables con aportes consistentes tanto en MI como en correlaciones moderadas, o con fundamento fisiológico claro, como Magnesium y HCO₃_Venous, completando así un conjunto de predictores compactos y representativos. En conjunto, estas quince variables ofrecen una base sólida para avanzar hacia etapas posteriores de modelización o análisis clínico.

#### 2.3.1.3 Pair plots para detectar visualmente valoes atípicos y potenciales relaciones entre variables

In [ ]:
columns_to_keep = ['LDH','pO2_Arterial','Hemoglobin','AGE','IL6','Glucose','HCO3_Arterial','AST','Troponin_I','Protein','Albumin','ALT','O2_Saturation','Magnesium','HCO3_Venous','Id','GENDER','INR']
df_patients_observations_medications_INR = df_patients_observations_medications_INR[columns_to_keep]

In [ ]:
# --- B. Preparación para el Ploteo en Subgrupos ---
target_col = 'INR'

# 3. Crear INR_Group
bins = [
    df_patients_observations_medications_INR[target_col].min() - 1,
    2.0,
    3.0 + 1e-6,
    df_patients_observations_medications_INR[target_col].max() + 1
]

df_patients_observations_medications_INR['INR_Group'] = pd.cut(
    df_patients_observations_medications_INR[target_col],
    bins=bins,
    labels=['Bajo (<2.0)', 'Normal (2.0-3.0)', 'Alto (>3.0)'],
    right=False,
    include_lowest=True
).astype('category')

# 4. Obtener variables numéricas
plot_features = df_patients_observations_medications_INR.select_dtypes(include=np.number).columns.tolist()
if 'Id' in plot_features:
    plot_features.remove('Id')

# Usar todas las variables en un solo plot
chunks = [plot_features]

# 6. Configuración visual
colors = ["#2ecc71", "#f39c12", "#e74c3c"]
hue_order = ['Bajo (<2.0)', 'Normal (2.0-3.0)', 'Alto (>3.0)']

sns.set(font_scale=0.7)

threshold_display = 0.10

# --- C. Generar y guardar el Plot ---

print(f"Número de Pair Plots a generar: {len(chunks)}")
print(f"Variables totales a plotear: {len(plot_features)}\n")

for i, chunk in enumerate(chunks):
    plot_df = df_patients_observations_medications_INR[chunk + ['INR_Group']]
    
    ax = sns.pairplot(
        plot_df,
        hue='INR_Group',
        palette=colors,
        hue_order=hue_order,
        height=1.2,
        aspect=1.0,
        kind="scatter",
        diag_kind="kde"
    )

    title = f'Pair Plot {i+1} de {len(chunks)}: Variables con |r| > {threshold_display}'
    ax.fig.suptitle(title, size=14, y=1.03)

    # ------------------------------------------------------------------
    # Guardar figura
    filename = f"pairplot_{i+1}_variables.png"
    ax.fig.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Guardado: {filename}")
    # ------------------------------------------------------------------

    plt.show()


A partir del análisis visual del pair plot, se observa que los pacientes clasificados como de mayor gravedad —especialmente aquellos en el grupo de alto riesgo o que no sobrevivieron— tienden a mostrar valores más extremos en varias variables clínicas clave. Destacan particularmente las elevaciones en INR y bilirrubina total, que parecen marcar una alteración sistémica más pronunciada y diferenciar con claridad a estos grupos respecto a los pacientes de bajo riesgo. Del mismo modo, la saturación de oxígeno revela una separación evidente: los pacientes con mejor pronóstico se concentran en valores altos y estables, mientras que los grupos críticos presentan mayor dispersión y caídas significativas. En contraste, variables como el hematocrito o la pO₂ arterial no muestran patrones tan definidos, y sólo la relación esperada entre la presión sistólica y diastólica mantiene una correlación clara. En conjunto, el gráfico sugiere que el riesgo clínico no depende de una sola medida, sino de un perfil en el que los marcadores hepáticos y la oxigenación desempeñan un papel central para distinguir entre estabilidad y deterioro.

#### 2.3.1.4. Distribución de frecuencias

In [ ]:
plt.figure(figsize=(20, 6))  # Ajusta los valores según lo grande que lo quieras

sns.set(font_scale=1.2)
sns.countplot(x="INR", data=df)

plt.ylabel("Data count")
plt.xlabel("")
plt.title("Target distribution")

plt.show()


In [ ]:
# 3. Crear la columna INR_Group (Necesaria para el 'hue' del pair plot)
sns.set(font_scale = 1)
sns.countplot(x="INR_Group", data=df).set(ylabel = "Data count"
            , xlabel = "")
plt.title('Target distribution')

print('--- Distribución de la Columna Target (INR_Group) ---')
conteo_inr = df['INR_Group'].value_counts()
print(conteo_inr)

#### 2.3.2 Categorical variables

In [ ]:
categorical = ['GENDER']

plt.figure(figsize=(8, 6))
sns.set(font_scale = 1.5)

sns.countplot(
    data=df,
    x='GENDER',
    hue='INR_Group',   # Usar la variable categórica de interés
    palette='Set2'
)

plt.title("Distribución de GENDER según INR_Group")
plt.xlabel("Género")
plt.ylabel("Conteo")
plt.tight_layout()
plt.show()


In [ ]:
#df['GENDER'].value_counts()
#pd.crosstab(df['GENDER'], df['INR_Group'])

df['GENDER'].value_counts()
df['GENDER'].value_counts(normalize=True)
gender_by_inr = pd.crosstab(df['INR_Group'], df['GENDER'], normalize='index')
print(gender_by_inr)

In [ ]:
#Guardamos el csv actualizado
df_patients_observations_medications_INR.to_csv('data/df_patients_observations_medications_INR.csv')

#### 2.3.3 Feature importance: conclusión

Las correlaciones más altas en las características numéricas son 'Oldpeak, y 'MaxHR' y 'ST_Slope', 'ExerciseAngina' y 'ChestPainType' parecen ser las características categóricas más relevantes.

Entrenaremos diferentes modelos durante la fase de entrenamiento (ver apartado 4. Entrenamiento del modelo) para seleccionar el que mejor desempeñe nuestra tarea. Cada modelo usa los datos a su modo. Podríamos verificar programáticamente el efecto de eliminar características en el rendimiento del modelo de cada uno de los modelos que entrenaremos. Si la presencia o eliminación de una característica no afecta los resultados, la característica no es relevante para ese modelo y podemos descartarla al ajustar los parámetros del modelo y para las fases de validación y prueba. Además, podríamos buscar características polinómicas que muestren colinealidades o aplicar métodos de reducción de dimensionalidad (Linear Discriminant Analysis, Principal Component Analysis (PCA),...). Veremos a continuación que el modelo de referencia inicial ya funciona razonablemente bien sin hacer feature engineering. Simplemente codificaremos las características categóricas y escalaremos (ver la siguiente sección 3. Preparación de datos). Por otro lado, si aplicamos un PCA a los datos (o un over- o under-sampling en el caso de datos no balanceados, como en SMOTE "Synthetic Minority Over-sampling Technique"), luego no sería fácil interpretar los resultados de explicabilidad y cuantificación de incertidumbre que explicamos en los siguientes Jupyter Notebooks.